# Using itertools

http://www.blog.pythonlibrary.org/2016/04/20/python-201-an-intro-to-itertools/

In [ ]:
"""Index positions and amounts of all combinations adding up to the target amount"""

from itertools import combinations

dif = 17000
amounts = [100000 , 65000 , 7000 , 3000 , 17000 , 10000 , 3000 , 4000]
amounts = [(idx, amount) for idx, amount in enumerate(amounts, start=1)]
totals = (comb for qty in range(2, len(amounts) + 1)
          for comb in combinations(amounts, qty)
          if sum(n[1] for n in comb) == dif
              )
for total in totals:
    print(*total, sep=", ")

In [1]:
#itertools

from itertools import cycle,islice, count, repeat, accumulate, compress, chain

for num in count(5):  # infinite counter
    if num == 10:  # need an exit plan
        break
    print(num, end="; ")
print()
for num in islice(count(5), 5):  # limits iterations
    print(num, end="; ")
print()
for letter in islice(cycle('xyz'), 5):
    print(f'{letter}', end="; ")
print()   
for string in repeat('Stuart', 5):  # boring repititions
    print(f'Hi {string}', end="; ")
print()
for num in accumulate(islice(count(0), 10)):
    print(num, end="; ")
print()

nums = [1,2,3]
names = 'fred', 'bob'
for thing in chain(nums, names):
    print(thing, end="; ")
print()

bools = [True, False, True]
for num in compress(nums, bools):
    print(num, end='; ')
print()

5; 6; 7; 8; 9; 
5; 6; 7; 8; 9; 
x; y; z; x; y; 
Hi Stuart; Hi Stuart; Hi Stuart; Hi Stuart; Hi Stuart; 
0; 1; 3; 6; 10; 15; 21; 28; 36; 45; 
1; 2; 3; fred; bob; 
1; 3; 


In [2]:
from itertools import combinations, filterfalse

def off_target(nums):  # combinations that don't equal target
    return not sum(n for _, n in nums) == dif

def output_matches(totals, amounts):
    print(f"\n\nAmounts that sum to {dif}:\n"
          f"  (from: {', '.join(str(n) for n in amounts)})\n")
    for total in totals:
        for idx, num in total:
            print(f'{idx:4}: {num:5}', end='  ')
        print()


difs = ((17000, [100000 , 65000 , 7000 , 3000 , 17000 , 10000 , 3000 , 4000]), # original
        (10000, [100000 , 65000 , 7000 , 3000 , 17000 , 10000 , 3000 , 4000]), # test
       )

for dif, amounts in difs:
    amounts_idx = [(idx, amount) for idx, amount in enumerate(amounts, start=1)]
    totals = (comb for qty in range(2, len(amounts) + 1)
                      for comb in filterfalse(off_target,
                            (combinations(amounts_idx, qty))))
    output_matches(totals, amounts)



Amounts that sum to 17000:
  (from: 100000, 65000, 7000, 3000, 17000, 10000, 3000, 4000)

   3:  7000     6: 10000  
   4:  3000     6: 10000     8:  4000  
   6: 10000     7:  3000     8:  4000  
   3:  7000     4:  3000     7:  3000     8:  4000  


Amounts that sum to 10000:
  (from: 100000, 65000, 7000, 3000, 17000, 10000, 3000, 4000)

   3:  7000     4:  3000  
   3:  7000     7:  3000  
   4:  3000     7:  3000     8:  4000  


# RPN Calculator using Class

https://www.reddit.com/r/learnpython/comments/fhk6nx/any_way_to_define_a_dictionary_of_methods_within/

In [ ]:
class Calculator:

    GUIDE = ''  # placeholder for init

    def __init__(self):
        self.clear()
        self.degrees = True
        self.scimode = False
        self.rpn = True
        self.msg = "h(elp) q(uit)"
        self.err = ""
        self.prompt = "> "
        print(Calculator.GUIDE)

    def __str__(self):
        elements = [self.err, self.msg, f'{self.stack}', self.prompt]
        self.display = ' | '.join(elements)
        return self.display

    @staticmethod
    def is_quit(text):
        return text.lower() in ('q', 'quit', 'exit', 'x')

    def stack_op(self, text):
        if text.lower() in Calculator.OPS:
            self.stack.append(Calculator.OPS[text.lower()][-1])
            return True
        return False

    def stack_num(self, text):
        num = None
        try:
            num = float(text)
            num = int(text)
        except ValueError:
            pass
        if num is None:
            return False
        self.stack.append(num)
        return True                

    def clear(self):
        self.display = ""
        self.memory = 0
        self.stack = []

    def clear_error(self):
        self.err = ""

    def set_msg(self, msg=""):
        self.msg = msg

    def enough_nums(self, required=2):
        """confirm if stack holds sufficient numbers for an operation"""
        if len(self.stack) >= required:
            for n in range(required):
                if not isinstance(self.stack[-1-n], (int, float)):
                    return False
            return True
        return False    

    def entry(self):
        while True:
            response = input(f'{self}')
            self.clear_error()
            if self.is_quit(response):
                return None
            elif self.stack_op(response):
                return True
            elif self.stack_num(response):
                return False
            self.err = 'ERR (unknown)'

    def op_add(self):
        if self.enough_nums():
            print(self.stack)
            self.stack.append(self.stack.pop() + self.stack.pop())
        else:
            self.err = f'ERR (not enough nums)'

    def op_sub(self):
        if self.enough_nums():
            self.stack.append(- self.stack.pop() + self.stack.pop())
        else:
            self.err = f'ERR (not enough nums)'

    def op_mul(self):
        if self.enough_nums():
            self.stack.append(self.stack.pop() * self.stack.pop())
        else:
            self.err = f'ERR (not enough nums)'

    def op_div(self):
        if self.enough_nums():            
            if not self.stack[-1] == 0:
                self.stack.append(self.stack.pop(-2) / self.stack.pop())
            else:
                self.err = f'ERR (div by 0 not allowed) {self.stack}'
        else:
            self.err = f'ERR (not enough nums)'

    def op_del1(self):
        if self.stack:
            self.stack.pop()

    def op_whole(self):
        if self.enough_nums(1):
            self.stack.append(int(self.stack.pop()))
        else:
            self.err = f'ERR (not enough nums)'

    def op_round(self):
        if self.enough_nums():
            self.stack.append(round(self.stack.pop(-2), self.stack.pop()))
        else:
            self.err = f'ERR (not enough nums)'

    def op_swap(self):
        if self.enough_nums(2):
            self.stack.extend((self.stack.pop(), self.stack.pop()))
        else:
            self.err = f'ERR (not enough nums)'

    def op_help(self):
        print(Calculator.GUIDE)

    def work(self):
        while True:
            status = self.entry()
            if status is None:
                return
            elif status:
                op = self.stack.pop()
                op(self)

    OPS = {'+': ('add two numbers', op_add),
        '-': ('subtract top from next down', op_sub),
        '*': ('multipple two numbers', op_mul),
        '/': ('divide 2 down by top', op_div),
        'int': ('truncate top to integer', op_whole),
        'round': ('round 2 down to top decimal places', op_round), 
        'h': ('display hel', op_help),
        'del': ('remove top number', op_del1),
        'swap': ('swap top two numbers', op_swap),
        'q': ('quit Calculator', None),
        }

    DETAILS = ''.join(f'\t{code:5} {desc}\n' for code, (desc, _) in OPS.items()) 

    GUIDE = f'Welcome to our simple RPN Calculator\n\n' \
            f'Operators/Commands:\n{DETAILS}\n'


calc1 = Calculator()
calc1.work()

I realise you've been given a few options already, but a slightly different approach is to put the method names in the dict:

In [ ]:
class Calculator:
    method_aliases = {'+': 'add', 'del': 'del1'} # and so on

    def add(self):
        ...

    def del1(self):
        ...

    def get_method(self, string):
        return getattr(self, self.method_aliases[string])

calculator = Calculator()
calculator.get_method('+')() # this calls calculator.add()

This way you can populate the dict anywhere, regardless of whether the methods have been defined yet or not. This also plays nicely with subclasses. For example if you have a subclass that defines its own add method, that will be called instead of the parent class's method, without having to give the subclass a different method_aliases dict.

Though if you're going to do this kind of thing often, it may be nicer to do some magic with decorators and/or metaclasses to get a nice API like the following:

In [ ]:
@alias_class
class Calculator:
    @alias('+')
    def add(self):
        ...

calculator = Calculator()
calculator['+']()

In [ ]:
x = [1,2,3]
x.extend([4,5,6])
x

There are a number of ways of coding this up, depending on the exact semantics you want.

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

p = np.linspace(0, 20, 100)
plt.plot(p, np.sin(15*p))
plt.show()



In [ ]:
    density5_01 = np.array([ 0. , 0. , 0. , 0. ,
                    0. , 0. , 0. , 0. ,
                    0. , 0. , 30282.53606145, 26502.46914671,
                    0. , 0. , 0. , 33493.91806271,
                    15153.27538047])
    mask2 = density5_01 == 0
    print(mask2)

In [ ]:
test[:] == 0

In [ ]:
    shapes = {"square": 35, "circle": 35, "rectangle": 23}
    shape = 'square'
    if shape in shapes:
        print(f'The cost of square is: ${shapes["square"]:.2f}')

In [ ]:
def permutations(string):
    result = set([string])
    if len(string) == 2:
        result.add(string[1] + string[0])
    elif len(string) > 2:
        for i, c in enumerate(string):
            for s in permutations(string[:i] + string[i+1:]):
                result.add(c+s)
    return list(result)

print(permutations('abcd'))

How do you write a Python program if you are given three numbers a, b, and c? Write a program to determine the largest number that is less than or equal to c and leaves a remainder b when divided by a

In [ ]:
def get_num(msg):
    while True:
        try:
            num = int(input(msg))
            break
        except ValueError:
            pass
        print('Integer number, please')
    return num
c = get_num('Limit? ')
a = get_num('Divisor? ')
b = get_num('Remainder? ')

for num in range(c, b, -1):
    if num % a == b:
        print(f'{num} is largest number <= {c} divisible by {a} leaving {b}.')
        break
else:
    print(f'Can\t fine a number <= {c} divisible by {a} leaving {b}.')

In [ ]:
    xs=range(10)
    %timeit map(hex, xs)
    %timeit (hex(x) for x in xs)
    %timeit map(lambda x: x+2, xs)
    %timeit (x+2 for x in xs)

In [ ]:

    229 ns ± 4.84 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
    387 ns ± 6.72 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
    268 ns ± 3.11 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
    377 ns ± 3.77 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)

In [ ]:
    x = input("Enter numbers separated by comma ")
    new_list = x.split(",") # note the use of split
    sample_list = []
    for item in new_list:
        sample_list.append(int(item))
    average = sum(sample_list) / (len(sample_list))
    print(average)

In [ ]:
    *trailing_qtrs, current_qtr = sales_record
    trailing_avg = sum(trailing_qtrs) / len(trailing_qtrs)
    return avg_comparison(trailing_avg, current_qtr)

In [ ]:
    def avgComparison(salesRecord):
        *trailingQtrs, currentQtr = salesRecord
        trailingAvg = sum(trailingQtrs) / len(trailingQtrs)
        return int(trailingAvg), currentQtr
    sales_record = [10, 8, 7, 1, 9, 5, 10, 3]
    print(avgComparison(sales_record))

In [ ]:
    import csv

    # test data
    data = """"Scott, Mary", 1300, food, lunch
    "Potts, Kev", 0730, porridge, breakfast 
    "Smith, Paul", 2000, pulses, protein 
    "Carol, Wendy", 2100, gin, relaxing"""
    # written to file
    filename1 = 'in_file.csv'
    with open(filename1, 'w') as f:
        f.writelines(data)

    readCSV = csv.reader(StringIO(data), delimiter=',')
    with open(filename1) as in_file, open(filename2, "w") as out_file:
        readCSV = csv.reader(in_file, delimiter=',')
        for line in readCSV:  # line references a list object
            name, time, item, something = line
            out_file.write(f'Hello {name}, right now is {time}. You need {item} for {something}\n')




In [ ]:
cat csvtest.csv

In [ ]:
    user_input = input("Enter a list of integers: ")
    bounds = input("Enter the upper and lower bounds: ")
    inBounds = []
    try:
        lower, upper = [int(num) for num in bounds.split()]
        inBounds = [num for num in
                            (int(string) 
                                for string in user_input.split()
                            )
                        if lower <  num < upper]
    except ValueError as e:
        print(f'invalid inputs: {e}')
    else:
        print(*inBounds, sep=", ")

In [ ]:
    character_name = "Tom"
    character_age = 50.5678213
    is_male = True
    print(f"There once was a man named {character_name}, ")
    print(f"he was {round(character_age)} years old. ")

    character_name = "Mike"
    print(f"He really liked the name {character_name}, ")
    print(f"but he didn't like being {round(character_age)} years old.")

In [ ]:
    data =  ([1, 2, 3, 4,], [55, 34, 3, 19,]), \
            ([3, 4, 5, 6,], [12, 21, 67, 1,]),
            
    ddata = dict(zip(*data[0]))
    for key, value in zip(*data[1]):
        if key in data:
            if value < ddata[key]:
                continue
        ddata[key] = value

    print(ddata)
    new_data = list(ddata.keys()), list(ddata.values())
    print(new_data)

In [ ]:
list(ddata.keys()), list(ddata.values())

In [ ]:
new_data = [ ]

In [3]:
# testing